In [1]:
import pandas as pd
import zipfile
import xml.etree.ElementTree as ET
import os
import requests

api_key = '0728a6d81b70b86763fd38daafcc1fccff0beedb'

# 기업 고유번호 가져오는 방법
# 개발가이드: https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018
# 기업 고유번호 XML 파일 다운로드 URL
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'

# XML 파일 다운로드
response = requests.get(url)
zip_path = 'corpCode.zip'

with open(zip_path, 'wb') as file:
    file.write(response.content)

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# XML 파일 파싱
xml_file = 'CORPCODE.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

# 삼성전자의 고유번호 찾기
corp_name = '삼성전자'
corp_code = None

for company in root.findall('list'):
    if company.find('corp_name').text == corp_name:
        corp_code = company.find('corp_code').text
        break

if corp_code:
    print(f"삼성전자의 고유번호: {corp_code}")
else:
    print("삼성전자를 찾을 수 없습니다.")

# 사용이 끝난 파일 삭제
os.remove(zip_path)
os.remove(xml_file)

# 기업 개황 정보 조회
# DART API 엔드포인트
url = 'https://opendart.fss.or.kr/api/company.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': corp_code,
}

# API 요청
response = requests.get(url, params=params)
data = response.json()

# 응답 상태 확인
if data.get('status') == '000':  # 성공 상태 코드
    # 데이터프레임으로 변환하여 출력
    disclosures = pd.DataFrame([data])
    print(disclosures.head())
else:
    print("Error:", data.get('message'))

# 재무 정보 조회
# DART API 엔드포인트 (재무정보 지표)
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': corp_code,
    'bsns_year': '2024',
    'reprt_code': '11013',  # 1분기 보고서 코드
    'fs_div': 'CFS'  # 연결재무제표 (CFS)
}

# API 요청
response = requests.get(url, params=params)
data = response.json()

# 응답 상태 확인
if data.get('status') == '000':  # 성공 상태 코드
    balancesheet = pd.DataFrame(data['list'])
    print(balancesheet.head())
else:
    print("API 오류:", data.get('message'))


삼성전자의 고유번호: 00126380
  status message corp_code corp_name                corp_name_eng stock_name  \
0    000      정상  00126380   삼성전자(주)  SAMSUNG ELECTRONICS CO,.LTD       삼성전자   

  stock_code ceo_nm corp_cls       jurir_no     bizr_no  \
0     005930    한종희        Y  1301110006246  1248100998   

                        adres               hm_url ir_url        phn_no  \
0  경기도 수원시 영통구  삼성로 129 (매탄동)  www.samsung.com/sec         02-2255-0114   

         fax_no induty_code    est_dt acc_mt  
0  031-200-7538         264  19690113     12  
         rcept_no reprt_code bsns_year corp_code stock_code fs_div   fs_nm  \
0  20240516001421      11013      2024  00126380     005930    CFS  연결재무제표   
1  20240516001421      11013      2024  00126380     005930    CFS  연결재무제표   
2  20240516001421      11013      2024  00126380     005930    CFS  연결재무제표   
3  20240516001421      11013      2024  00126380     005930    CFS  연결재무제표   
4  20240516001421      11013      2024  00126380     005930    C

In [6]:
import dart_fss as dart_fss
import pandas as pd

api_key = '0728a6d81b70b86763fd38daafcc1fccff0beedb'
dart_fss.set_api_key(api_key=api_key)

corp_list = dart_fss.get_corp_list()

corp_list.corps


[[00434003]다코,
 [00430964]굿앤엘에스,
 [00388953]크레디피아제이십오차유동화전문회사,
 [00179984]연방건설산업,
 [00420143]브룩스피알아이오토메이션잉크,
 [00401111]매경아이비아이,
 [00435534]캐드뱅크,
 [00430186]엠와이오피삼차유동화전문유한회사,
 [00430201]엠와이오피이차유동화전문유한회사,
 [00430210]엠와이오피일차유동화전문유한회사,
 [00430229]포스미디어,
 [00140432]축복할렐루야,
 [00426208]한국전자화학,
 [00433262]ConnachtCapitalMarketInvestmentLtd.,
 [00433749]선진아이티,
 [00433785]팀스코리아,
 [00196079]에넥스하이테크,
 [00435048]세이스텝바이스텝혼합형펀드,
 [00435057]유리알파헷지채권혼합형펀드,
 [00108843]한기술정보통신,
 [00443232]다움종합건설,
 [00248293]한국애치슨,
 [00455662]고은상사,
 [00149318]조흥종합건설,
 [00451347]유니즈유통,
 [00415105]창대화장품,
 [00420824]문화종합건설,
 [00389439]한국종합미디어,
 [00454016]아이펜텍,
 [00452072]대홍염공,
 [00437602]지오항공여행사,
 [00365518]헤이아니타코리아,
 [00246967]태산산업개발,
 [00445559]군포종합시장,
 [00444657]기우,
 [00448936]라시도,
 [00405658]예전미디어,
 [00410818]세미가,
 [00416061]대닉스,
 [00448909]인포핸드,
 [00445407]지현개발,
 [00421647]한국부동산신탁제일차유동화전문유한회사,
 [00421425]ISCCaymanLtd.,
 [00423265]연산,
 [00423292]씨케이알유동화전문유한회사,
 [00420602]디에스피,
 [00307505]LOMBARDKOREAILIMITED,
 [00241397

In [10]:
all = dart_fss.api.filings.get_corp_code()
all[0]

df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]
df_non_listed = df[df['stock_code'].isnull()]

df_listed.head()

,corp_code,corp_name,stock_code,modify_date
1947,00260985,한빛네트,036720,20170630
1959,00264529,엔플렉스,040130,20170630
1960,00358545,동서정보기술,055000,20170630
2703,00231567,애드모바일,032600,20170630
3778,00359614,리더컴,056140,20170630


In [11]:
df_listed.count()

corp_code      3793
corp_name      3793
stock_code     3793
modify_date    3793
dtype: int64

In [12]:
def get_salary(name):
    data = dart_fss.api.info.emp_sttus(corp_code, '2021', '11011')

    df = pd.DataFrame(data['list'])
    df = df[['corp_name', 'sexdstn', 'jan_salary_am']]

    df_result = pd.DataFrame()
    doc = {
        '기업명': df.iloc[0, 0],
        '연봉(남)': df[df['sexdstn'] == '남'].iloc[0, 2],
        '연봉(여)': df[df['sexdstn'] == '여'].iloc[0, 2]
    }
    df_result = df_result.append(doc, ignore_index=True)

    df_result['연봉(남)'] = pd.to_numeric(df_result['연봉(남)'].str.replace(',', ''))
    df_result['연봉(여)'] = pd.to_numeric(df_result['연봉(여)'].str.replace(',', ''))

    return df_result


In [18]:
corp_codes = list(df_listed.sample(50)['corp_code'])

dfs = []
for corp_code in corp_codes:
    try:
        df = get_salary(corp_code)
        dfs.append(df)
    except:
        print(f'error - {corp_code}')

df_result = pd.concat(dfs)

df_result['차이(남-여)'] = df_result['연봉(남)'] - df_result['연봉(여)']
df_result['평균'] = (df_result['연봉(남)'] + df_result['연봉(여)']) / 2

df_result.sort_values(by='차이(남-여)', ascending=True)


C:\TempFolder\ipykernel_22844\3387255517.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(doc, ignore_index=True)
C:\TempFolder\ipykernel_22844\3387255517.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(doc, ignore_index=True)


error - 00248691


C:\TempFolder\ipykernel_22844\3387255517.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(doc, ignore_index=True)


error - 00757816
error - 01370863


C:\TempFolder\ipykernel_22844\3387255517.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(doc, ignore_index=True)


...

error - 00823687
error - 01624162
error - 00264787


C:\TempFolder\ipykernel_22844\3387255517.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(doc, ignore_index=True)


error - 01689266


C:\TempFolder\ipykernel_22844\3387255517.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(doc, ignore_index=True)


,기업명,연봉(남),연봉(여),차이(남-여),평균
0,유바이오로직스,51526000,58786000,-7260000,55156000.0
0,아스트,42000000,49000000,-7000000,45500000.0
0,미애부,37000000,38000000,-1000000,37500000.0
0,희림,66000000,60000000,6000000,63000000.0
0,유진기업,70000000,64000000,6000000,67000000.0
0,티앤엘,43000000,37000000,6000000,40000000.0
0,신라젠,70000000,63000000,7000000,66500000.0
0,F&F 홀딩스,37101000,29944000,7157000,33522500.0
0,스맥,47022000,38122000,8900000,42572000.0
0,위세아이텍,44653000,35602000,9051000,40127500.0
